In [71]:
import os

# import IPython.display as ipd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as skl
import sklearn.utils, sklearn.preprocessing, sklearn.decomposition, sklearn.svm
import librosa
import librosa.display
# import get_related_artists as from_echo
import re
import pickle
from IPython.display import display, HTML
import python_utils as utils


In [72]:
# Load metadata and features.
tracks = pd.read_csv('fma_metadata/tracks.csv')
r_tracks = pd.read_csv('fma_metadata/raw_tracks.csv')
genres = pd.read_csv('fma_metadata/genres.csv')
features = pd.read_csv('fma_metadata/features.csv')
echonest = pd.read_csv('fma_metadata/echonest.csv')
r_artists = pd.read_csv('fma_metadata/raw_artists.csv')

tracks.shape, genres.shape, features.shape, echonest.shape

((106576, 53), (163, 5), (106577, 519), (13132, 250))

In [81]:
tracks_sm = tracks.loc[(tracks['set.1'] == 'small') & (tracks['set'] == 'training')]

In [82]:
# Spotify  Business

import sys
import spotipy
import pprint
''' shows recommendations for the given artist
'''


from spotipy.oauth2 import SpotifyClientCredentials
client_credentials_manager = SpotifyClientCredentials(client_id='ba6790bdcd434f06b7b577e344c6e0ae', client_secret='145715e565ff48469c306484896a34f5')
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
sp.trace=False

def get_artist(name):
    results = sp.search(q='artist:' + name, type='artist')
    items = results['artists']['items']
    if len(items) > 0:
        return items[0]
    else:
        return None


def get_related(uri):
    
    related = sp.artist_related_artists(uri)
    ans = [] 
    
    for artist in related['artists']:
        ans.append(artist['name'])
#        print artist['name']
        
    
    return ans
    
def save_spot_rel_artists(name, table):
    with open('saved_data/%s.pickle'%name, 'wb') as writer:
        pickle.dump(table, writer, protocol=pickle.HIGHEST_PROTOCOL)
        
def load_spot_rel_artists(name):
    with open(name, 'rb') as handle:
        thing = pickle.load(handle)   
    return thing

In [83]:
len(tracks_sm['artist.12'].unique())

1885

In [76]:
# get spotify artist_id for every artist, and related artists
na = 0
artist_rel = {}
artists_sm = tracks_sm['artist.12'].unique()

for i, elem in enumerate(artists_sm):
    
    artist = re.split(',', elem)[0]
    if i%100 == 0:
        print i
    
#     a_id = from_echo.get_artist(artist)
    try:
        a_id = get_artist(artist)

        if a_id:
    #         related_artists = from_echo.get_related(a_id['uri'])
            related_artists = get_related(a_id['uri'])
    #        print related_artists
            if related_artists:
                artist_rel[artist] = related_artists
    #             artist_rel[artist] = a_id
            else:
                na += 1
        else:
            na += 1
    except:
        continue
        



0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
retrying ...1secs
1500
1600
retrying ...1secs
1700
retrying ...1secs
retrying ...1secs
1800
1900
2000
2100
retrying ...1secs
2200
retrying ...1secs
retrying ...1secs
2300
2400
retrying ...1secs
retrying ...1secs
2500
2600
retrying ...1secs
retrying ...2secs
2700
2800
retrying ...1secs
retrying ...1secs
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
retrying ...1secs
4800
retrying ...1secs
4900
5000
5100
5200
5300
5400
5500
retrying ...1secs
5600
5700
retrying ...1secs
5800
retrying ...1secs
5900
6000
6100
retrying ...1secs
6200
6300
6400
retrying ...1secs
6500
6600
6700
6800
retrying ...1secs
6900
7000
7100
retrying ...1secs
7200
7300
7400
retrying ...1secs
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11

In [77]:
# save_spot_rel_artists("artist_rel_medium_training", artist_rel)
save_spot_rel_artists("artist_rel_    _training", artist_rel) 

In [139]:
# load artists
artist_rel = load_spot_rel_artists("saved_data/artist_rel_small_training.pickle")

In [141]:
len(artist_rel)

1026

In [142]:
# Subsetting Business, Creating Master tables

# Creating Condensed Artist Map, and Master 2D Table
artist_all = {}

for key in artist_rel:
    artist_all[key] = set()

for key in artist_rel:
    for elem in artist_rel[key]:
        if artist_rel.get(elem, None) is not None:
            artist_all[key].add(elem)
            artist_all[elem].add(key)

# assert len(artist_all) == len(artist_rel)  
# deleting artists with no related artist in set
keys = artist_all.keys()
for key in keys:
    if not artist_all[key]:
        del artist_all[key]
#         pass
    else:
        artist_all[key] = list(artist_all[key])
        
# Artist Index Map
artist_index = {}
for i, a in enumerate(artist_all.keys()):
    artist_index[a] = i


In [143]:
artist_rel_master = np.zeros((len(artist_all.keys()), len(artist_all.keys())))

# Master table of artist relation
for i, a in enumerate(artist_all.keys()):
    # going through each artist
    for b in artist_all[a]:
        artist_rel_master[artist_index[a]][artist_index[b]] = 1

# print artist_rel_master[artist_index['Robo']][artist_index['Robo']]

In [144]:
# Master table of artists per genre

# songs per genre
genres_all = {} # maybe redundant
c = 0
artists_set = list(tracks_sm['artist.12'])
genres_artist = {}
i = 0
for gens in tracks_sm['track.9']:
    l = [re.split(']', s) for s in re.split(', |\[', gens)]
    gens = [item for sublist in l for item in sublist]
    for g in gens:
        if g.isdigit():
            if genres_all.get(int(g), -1) != -1:
                pass
            else:
                genres_all[int(g)] = c
                c += 1
            if genres_artist.get(int(g), -1) == -1:
                genres_artist[int(g)] = set()
            if artist_all.get(artists_set[i], None) is not None:
                genres_artist[int(g)].add(artists_set[i]) 
    i += 1



In [145]:
# creating master table of artists per genre

genre_artist_master = np.zeros((len(genres_all.keys()), len(artist_all.keys())))

for g in genres_all.keys():
#     print genres_artist[g]
    for a in genres_artist[g]:
#         print a
        genre_artist_master[genres_all[g]][artist_index[a]] = 1
        
# print genre_artist_master

In [146]:
# Number of songs per artist
songs_per_artist = {}
for k in artist_all.keys():
    songs_per_artist[k] = len(tracks_sm[tracks_sm['artist.12'] == k])

In [147]:
# Calculating D

def calc_D(songs_per_artist, num_artists_set):
    # total number of songs in Set
    num_songs_set = 0
    data = []
    for k in songs_per_artist:
        num_songs_set += songs_per_artist[k]
        if songs_per_artist[k] != 0:
            data.append(songs_per_artist[k])

    data = np.array(data)
    mn = np.mean(data, axis=0)
    sd = np.std(data, axis=0)
    
    D = 1 - (sd/(mn * (np.sqrt(num_artists_set) - 1)))

    return D


In [148]:
# Calculating C

def calc_C(num_artists_set, genres_artist):
    # total number of genres in Set
    sum_gen = genres_artist.sum(axis=1)
    num_gen_set = np.count_nonzero(sum_gen)

    mn = np.mean(sum_gen, axis=0)
    sd = np.std(sum_gen, axis=0)

    C = 1 - (sd/(mn * (np.sqrt(len(sum_gen)) - 1)))

    return C

In [149]:
# Calculating S

def calc_S(artists_in_set, songs_per_artist):
     
    num_edges = np.count_nonzero(artists_in_set)/2

    # total number of artists in Set
    num_artists_set = len(songs_per_artist.keys())

    # std dev of artist connections per artist
    a_edges = artists_in_set.sum(axis=1)
    mn = np.mean(a_edges, axis=0)
    sd = np.std(a_edges, axis=0)

    S = 1 - (sd/(mn * (np.sqrt(num_artists_set) - 1)))
    return S

In [150]:
len(artist_all)

394

In [151]:
def main_setup(N):
# Main Loop of Calculation
    import copy
    # N is number of Artists in Set

    global artist_all, genre_artist_master, artist_rel_master, artist_index, songs_per_artist
    
    # N = 200    
    artists_in_set = np.random.choice(artist_all.keys(), N, replace=False)
    artists_not_in_set = artist_all.keys()
    for a in artists_in_set:
        artists_not_in_set.remove(a)

    len(artists_in_set), len(artists_not_in_set)

    # songs per artist subset    
    s_p_a = {}
    for a in artists_in_set:
        s_p_a[a] = songs_per_artist[a]

    # artists per genre subset
    num_genres, num_artists = genre_artist_master.shape
    g_a = np.zeros((num_genres, N))
    a_rel = np.zeros((N, N))
    index = 0

    ai = [artist_index[k] for k in artists_in_set]

    for i in range(num_genres):
        g_a[i, :] = genre_artist_master[i, ai]

    for a in artists_in_set:
        ind = artist_index[a]
        a_rel[index, :] = artist_rel_master[ind, ai]
        a_rel[:, index] = artist_rel_master[ai, ind]    
        index +=1
        
    return a_rel, g_a, s_p_a, artists_in_set, artists_not_in_set, ai

In [163]:

def main_run(N):

    global g_a, s_p_a, a_rel, artists_in_set, artists_not_in_set, songs_per_artists, genre_artist_master, artist_rel_master, artist_index, ai
    
    best_artist = []

    C = calc_C(N, g_a)
    D = calc_D(s_p_a, N)
    S = calc_S(a_rel, s_p_a)
    goodness = C * D * S

    
    g_list = []
    best_set = []
    g_list_local = []
    g_list_local.append(goodness)
    c_list_local = []
    c_list_local.append(C)
    d_list_local = []
    d_list_local.append(D)
    s_list_local = []
    s_list_local.append(S)
    

    for epoch in range(3):


        # main loop for optimization
        for i, a in enumerate(artists_in_set):
            #remove an artist
            s_p_a[a] = 0
            g_a[:, i] = 0
            a_rel[i, :] = 0
            a_rel[:, i] = 0

            change = 0
            local_goodness = - float('inf')
            local_c = 0.
            local_d = 0.
            local_s = 0.
            # add artist from other set one at a time
            for b in artists_not_in_set:

                # add from other set
                s_p_a[b] = songs_per_artist[b]
                ai[i] = artist_index[b]
                g_a[:, i] = genre_artist_master[:, artist_index[b]]
                a_rel[i, :] = artist_rel_master[artist_index[b], ai]
                a_rel[:, i] = artist_rel_master[ai, artist_index[b]]

                Cee = calc_C(N, g_a)
                Dee = calc_D(s_p_a, N)
                Sss = calc_S(a_rel, s_p_a)

                G = Cee * Dee * Sss
                if G > local_goodness:
                    local_goodness = G
                    change = b
                    local_c = Cee
                    local_d = Dee
                    local_s = Sss

                # delete other element
                s_p_a[b] = 0
                g_a[:, i] = 0
                a_rel[i, :] = 0
                a_rel[:, i] = 0       

    #         print goodness
            if local_goodness > goodness:
                goodness = local_goodness
                C = local_c
                D = local_d
                S = local_s
                ai[i] = artist_index[change]
                s_p_a[change] = songs_per_artist[change]
                g_a[:, i] = genre_artist_master[:, artist_index[change]]
                a_rel[i, :] = artist_rel_master[artist_index[change], ai]
                a_rel[:, i] = artist_rel_master[ai, artist_index[change]]
                artists_not_in_set.append(a)
                artists_not_in_set.remove(change)
            else:
                ai[i] = artist_index[a]
                s_p_a[a] = songs_per_artist[a]
                g_a[:, i] = genre_artist_master[:, artist_index[a]]
                a_rel[i, :] = artist_rel_master[artist_index[a], ai]
                a_rel[:, i] = artist_rel_master[ai, artist_index[a]]



            g_list_local.append(goodness)
            c_list_local.append(C)
            d_list_local.append(D)
            s_list_local.append(S)


        g_list.append(np.mean(np.array(g_list_local), axis=0))
#         print epoch, g_list_local[-1]
    
    return g_list_local, c_list_local, d_list_local, s_list_local

In [156]:
def save_best(name, table):
    with open('saved_data/%s.pickle'%name, 'wb') as writer:
        pickle.dump(table, writer, protocol=pickle.HIGHEST_PROTOCOL)

In [165]:
# main run

#to be saved
overall_best_values = []
for i in range(20):
    print i
    list_of_best_values = []
    for n in range(50, 390, 10):
        print "N = ", n
        a_rel, g_a, s_p_a, artists_in_set, artists_not_in_set, ai = main_setup(n)
        v, c, d, s = main_run(n)
        list_of_best_values.append([n, v[-1], c[-1], d[-1], s[-1]])
#         print v[-1], c[-1], d[-1], s[-1]
    overall_best_values.append(list_of_best_values)

save_best("overall_GCSD_small_train_20", overall_best_values)

0
N =  50
N =  60
N =  70
N =  80
N =  90
N =  100
N =  110
N =  120
N =  130
N =  140
N =  150
N =  160
N =  170
N =  180
N =  190
N =  200
N =  210
N =  220
N =  230
N =  240
N =  250
N =  260
N =  270
N =  280
N =  290
N =  300
N =  310
N =  320
N =  330
N =  340
N =  350
N =  360
N =  370
N =  380
1
N =  50
N =  60
N =  70
N =  80
N =  90
N =  100
N =  110
N =  120
N =  130
N =  140
N =  150
N =  160
N =  170
N =  180
N =  190
N =  200
N =  210
N =  220
N =  230
N =  240
N =  250
N =  260
N =  270
N =  280
N =  290
N =  300
N =  310
N =  320
N =  330
N =  340
N =  350
N =  360
N =  370
N =  380
2
N =  50
N =  60
N =  70
N =  80
N =  90
N =  100
N =  110
N =  120
N =  130
N =  140
N =  150
N =  160
N =  170
N =  180
N =  190
N =  200
N =  210
N =  220
N =  230
N =  240
N =  250
N =  260
N =  270
N =  280
N =  290
N =  300
N =  310
N =  320
N =  330
N =  340
N =  350
N =  360
N =  370
N =  380
3
N =  50
N =  60
N =  70
N =  80
N =  90
N =  100
N =  110
N =  120
N =  130
N =  140
N = 

In [159]:
save_best("overall_GCSD_small_train_2", overall_best_values)

In [158]:
overall_best_values

[[[50,
   0.76788115289141168,
   0.85724123537372998,
   0.92600722909043687,
   0.96733427836185348],
  [100,
   0.76862527667810132,
   0.8639598775534918,
   0.91608570139275836,
   0.97114702386260698],
  [150,
   0.77430617888593845,
   0.86034412867054277,
   0.9211083512753,
   0.97707928465108373],
  [200,
   0.77234209742199911,
   0.85575044004049472,
   0.92667902251721246,
   0.97394233541637876],
  [250,
   0.76615306269177286,
   0.84867823207388116,
   0.93092005565692937,
   0.96975067864252207]],
 [[50,
   0.7628058906211439,
   0.85565398559995565,
   0.92297734446689894,
   0.96588366115549373],
  [100,
   0.77835987590784284,
   0.86577936104359221,
   0.92599923414022345,
   0.97087335883232995],
  [150,
   0.77789942866574946,
   0.85874915625538806,
   0.93112243640521863,
   0.97285997810949432],
  [200,
   0.76953181157986106,
   0.85496771588177289,
   0.92485774189735037,
   0.97319960867729649],
  [250,
   0.76583468116610887,
   0.8493585367702805,
   0.92

In [35]:
best_Ns = []

for best_values in overall_best_values:
    del best_values[0]
    best_n = max(best_values, key= lambda x: x[1])
    best_Ns.append(best_n)
    print best_n
        
save_best("best_n", best_Ns)

[60, 0.78111525798370773]
[150, 0.77954866252954913]
[120, 0.77993522934317239]
[110, 0.77646911766016335]
[100, 0.7783637953593896]
[130, 0.77926219205770808]
[170, 0.77595390929600838]
[130, 0.77927534882059624]
[130, 0.77837089018741989]
[160, 0.77985873843299269]


In [106]:
# find best N
best_n = max(list_of_best_values, key= lambda x: x[1])[0]    

NameError: name 'load_things' is not defined

In [107]:
best_n

[160, 0.77985873843299269]

In [131]:
# find starting and end points for best N for 20 points
st_pt = []
end_pt = []
for i in range(20):
    a_rel, g_a, s_p_a, artists_in_set, artists_not_in_set, ai = main_setup(n, artist_all, genre_artist_master, artist_rel_master, artist_index)
    v = main_run(N, g_a, s_p_a, a_rel, artists_in_set, artists_not_in_set, songs_per_artist, genre_artist_master, artist_rel_master, artist_index, ai)
    st_pt.append(v[0])
    end_pt.append(v[-1])

([0.65356870938280831,
  0.66606813672816767,
  0.64866463145636821,
  0.67480396744864479,
  0.64831049122515105,
  0.64826098224776318,
  0.65981049216326459,
  0.66099311901321589,
  0.642476769099668,
  0.64777515111703599,
  0.65578642618870531,
  0.63953530567651939,
  0.647303087683052,
  0.64969380903907237,
  0.62368621745752673,
  0.64089189389739198,
  0.64602399468385385,
  0.62356576138079822,
  0.6557206020386569,
  0.63032518568913765],
 [0.78069426800922037,
  0.78221230343918069,
  0.7879740306687476,
  0.78607502964824227,
  0.78493178046596079,
  0.78495336553512007,
  0.78235411140336342,
  0.78905053018872529,
  0.78353011423119301,
  0.78071054159265496,
  0.78341040909556037,
  0.78953767727371682,
  0.7830275934421268,
  0.78181276011080969,
  0.7824208871565006,
  0.78227219772663215,
  0.78211105300484784,
  0.7820047853743306,
  0.78423290280278091,
  0.78159199486138464])

In [132]:
def save_goodness(name, table):
    with open('saved_data/%s.pickle'%name, 'wb') as writer:
        pickle.dump(table, writer, protocol=pickle.HIGHEST_PROTOCOL)
        
save_goodness("best_st_pt", st_pt)
save_goodness("best_end_pt", end_pt)


NameError: name 'st_pt' is not defined